<a href="https://colab.research.google.com/github/vanshuwjoshi/Kaggle-Playground-S4E11/blob/main/Optuna_XGBoost_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

In [ ]:
data = pd.read_csv("/content/cleaned_v1.csv")
data.head()

,Work Pressure,CGPA,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Work/Study Hours,Financial Stress,Depression,Gender_Female,Gender_Male,Age_Group,City_encoded,Student,Profession_encoded,suicidal_thoughts,family_history
0,5.0,-1.0,2.0,8.0,0.0,2.0,1.0,2.0,0,1.0,0.0,3,0.192690,0,0.048567,0,0
1,4.0,-1.0,3.0,5.0,2.0,2.0,7.0,3.0,1,0.0,1.0,0,0.142206,0,0.055649,1,0
2,5.0,9.0,2.0,6.0,0.0,2.0,3.0,1.0,1,0.0,1.0,1,0.159196,1,0.585061,1,0
3,5.0,-1.0,1.0,5.0,1.0,2.0,10.0,1.0,1,0.0,1.0,0,0.131293,0,0.055649,1,1
4,1.0,-1.0,1.0,6.0,2.0,2.0,9.0,4.0,0,1.0,0.0,1,0.125739,0,0.056610,1,1


In [ ]:
data.drop(["Gender_Male"], axis=1, inplace=True)

In [ ]:
y = data["Depression"]
X = data.drop(["Depression"], axis=1)

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.4 MB/s eta 0:00:00


In [ ]:
import optuna

In [ ]:
def objective_all_metrics(trial):
    # Hyperparameter ranges
    param = {
        "objective": "binary:logistic",
        "eval_metric": "logloss",
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "n_estimators": trial.suggest_int("n_estimators", 300, 600),
        "max_depth": trial.suggest_int("max_depth", 2, 5),
        "subsample": 0.5,
        "colsample_bytree": 0.7,
    }

    # Cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies, roc_aucs, f1_scores = [], [], []

    for train_index, val_index in skf.split(X, y):
        X_strat_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_strat_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # Train XGBoost model
        model = XGBClassifier(**param, use_label_encoder=False)
        model.fit(
            X_strat_train,
            y_strat_train,
            eval_set=[(X_val, y_val)],
            verbose=False,
        )

        # Predictions and probabilities
        y_pred = model.predict(X_val)
        y_proba = model.predict_proba(X_val)[:, 1]

        # Calculate metrics
        accuracies.append(accuracy_score(y_val, y_pred))
        roc_aucs.append(roc_auc_score(y_val, y_proba))
        f1_scores.append(f1_score(y_val, y_pred))

    # Store metrics
    trial.set_user_attr("accuracy", sum(accuracies) / len(accuracies))
    trial.set_user_attr("roc_auc", sum(roc_aucs) / len(roc_aucs))
    trial.set_user_attr("f1_score", sum(f1_scores) / len(f1_scores))

    # Optimize for one metric (e.g., ROC-AUC)
    return sum(roc_aucs) / len(roc_aucs)

In [ ]:
# Optimize the study for ROC-AUC
study = optuna.create_study(direction="maximize")  # Maximize ROC-AUC
study.optimize(objective_all_metrics, n_trials=50)

[I 2024-11-24 04:54:06,821] A new study created in memory with name: no-name-4f04b474-04f5-4f1a-b309-deae6a3aeb7f
[I 2024-11-24 04:54:44,865] Trial 0 finished with value: 0.9740669113723213 and parameters: {'learning_rate': 0.1004864448364051, 'n_estimators': 435, 'max_depth': 4}. Best is trial 0 with value: 0.9740669113723213.
[I 2024-11-24 04:55:15,888] Trial 1 finished with value: 0.974445156451514 and parameters: {'learning_rate': 0.1413235723890479, 'n_estimators': 459, 'max_depth': 2}. Best is trial 1 with value: 0.974445156451514.
[I 2024-11-24 04:55:56,396] Trial 2 finished with value: 0.9744369927813841 and parameters: {'learning_rate': 0.08727733796230715, 'n_estimators': 519, 'max_depth': 3}. Best is trial 1 with value: 0.974445156451514.
[I 2024-11-24 04:56:29,139] Trial 3 finished with value: 0.9743765116804612 and parameters: {'learning_rate': 0.1931809990103323, 'n_estimators': 468, 'max_depth': 2}. Best is trial 1 with value: 0.974445156451514.
[I 2024-11-24 04:57:17,26

In [ ]:
# Display results
best_trial = study.best_trial
print(f"Best Parameters: {best_trial.params}")
print(f"Best ROC-AUC: {best_trial.value}")
print(f"Associated Accuracy: {best_trial.user_attrs['accuracy']}")
print(f"Associated F1-Score: {best_trial.user_attrs['f1_score']}")

Best Parameters: {'learning_rate': 0.074972461438185, 'n_estimators': 332, 'max_depth': 3}
Best ROC-AUC: 0.974519924406023
Associated Accuracy: 0.937867803837953
Associated F1-Score: 0.8256441365387035
